이번 글에서는 [벡터 연산](http://wikibootup.github.io/sicp/2-2-vector-operations.html)에 이어서 고차함수를(`map`과 같은) 이용하여 두가지 알고리즘을 구현하겠습니다.

### 호너의 규칙

$a_nx^n + a_{n-1}x^{n-1} + ... + a_1x + a_0$

위의 다항식은 호너의 규칙에 따라 아래와 같이 바꿀 수 있습니다.

$(...(a_nx + a_{n-1})x + ... + a_1)x + a_0$

(특이한 공식을 사용한 것은 아니고 공통 인수인 $x$로 묶어둔 것입니다.) 

이것을 함수적으로 바꿔보면,

```HornerEval x higherTerms
-> FirstTerm + x * HornerEval x (higherTerms+1)
-> FirstTerm + x * (SecondTerm + x * HornerEval x (higherTerms+1))
-> ...
```

`map`과 `accumulate` 함수를 이용해 이런 생각을 그대로 옮겨보면,

In [4]:
hornerEval :: (Num a, Foldable t) => a -> t a -> a
hornerEval x coeffSeqs = 
    foldl1  (\thisCoeff higherTerms -> x * thisCoeff + higherTerms)
            coeffSeqs

* 이전에 `Python`이나 `Scheme`(Git repo 상에 있는)에서는 `accumulate`라는 함수를 이용해서 리스트의 각 항에 필요한 연산을 적용했습니다. 여기서는 같은 기능인데 이름만 다른 `fold`라는 함수를 이용하였습니다.

아래의 경우로 테스트해보겠습니다.

1. 계수 고정 : $2^{15} + 2^{14} + ... + 2 + 1 = 2^{16}-1$

2. 홀수 항만 : $2^{15} + 2^{13} + ... + 2^3 + 2^1 = ?$

3. 짝수 항만 : $2^{14} + 2^{12} + ... + 2^2 = ?$

4. 계수 변화 : $1*2^{15} + 2*2^{14} + ... + 16 = ?$


* 편의상 $x=2$로 고정하겠습니다.

In [5]:
hornerEval 2 (replicate 16 1) == 2^16-1

True

In [6]:
hornerEval 2 (concat (replicate 7 [1,0]) ++ [0]) == 
    foldl (\acc x -> 2^x + acc) 0 [x*2 | x <- [1..7]] 

True

In [7]:
hornerEval 2 (concat (replicate 8 [0,1]) ++ [0]) == 
    foldl (\acc x -> 2^x + acc) 0 [x*2-1 | x <- [1..8]] 

True

In [8]:
hornerEval 2 [1..16] == 
    sum (zipWith (*) (map (\ x -> 2^x) (reverse [0 .. 15])) 
        [1..16])

True

### 퀸 퍼즐

퀸 퍼즐은 체스판에 퀸을 배열하는 문제입니다. 규칙은 다음과 같습니다.

1. 세로줄마다 퀸은 더도 덜도 말고 딱 1개 위치해야 한다.
2. 각 퀸은 서로 공격할 수 없는 위치에 있어야 한다.

( * 구현 방식을 이해하는 데 있어서 다음 링크에 나와 있는 시뮬레이션이 많은 도움이 되었습니다. https://www.cs.usfca.edu/~galles/visualization/RecQueens.html )

문제가 다소 복잡하므로 부분적인 기능부터 완전히 만들도록 하겠습니다.

먼저, 체스판 위에 새로운 퀸을 두려 할 때 그 위치에 놓을 수 있는지를 확인하는 기능을 구현하겠습니다.

```
이름 : isSafe
인자 : tryPos, posQueens
    tryPos : 새로운 퀸을 두려는 위치, 데이터 타입은 (a,a)
    posQueens : 이미 위치한 퀸의 위치, 데이터 타입은 [(a,a)]

```
퀸은 1. 수직, 2. 수평, 3. 대각선이면 어디든 갈 수 있습니다. 반대로 말하면, 방금 말한 세가지만 확인하면 안전한 위치인지 확인할 수 있습니다.

예를 들어, $0 < b <= a < n$인 $(a,b)$의 위치에 퀸을 두려한다고 하면, 다음 위치에 퀸이 없어야 합니다.

```
1. 수직 : (0,b), (1,b), ... , (n-1,b)
2. 수평 : (a,0), (a,1), ... , (a, n-1)
3. 하강하는 대각선 : (a-b, 0), (a-b+1, 1), ... , (n-1, n-(1+(a-b))
4. 상승하는 대각선 : a + b = x + y, [ (x,y) ∈ Position of the Queens ]
```

위를 통해, 수직과 수평에 대해서 확인을 할 때는 0부터 n-1의 위치까지 확인하면 됨을 알 수 있습니다.

하지만 여기서는 그럴 필요 없이 

1. 수직 : 열Column에 위치한 퀸이 있는지 확인
2. 수평 :행Row에 위치한 퀸이 있는지 확인

위와 같이 확인하도록 하겠습니다.

비슷하게 대각선에 대해서는 좌표를 이동하며 비교하지 않고 아래의 방법으로 확인하겠습니다.

1. 하강하는 대각선 : 각 퀸의 위치와 새로 두려는 위치에 `(a-b, 0)` 연산을 수행 후(이 때, $0 < b <= a < n$), 같은 경우가 있는지 확인
2. 상승하는 대각선 : 행Row과 열Column의 합이 같은 경우가 있는지 확인


* 이렇게 구현하면 체스판의 크기(n)와 무관하게 수직,수평,대각선을 확인할 수 있습니다. 따라서 함수의 기능을 파악하는게 한결 간단해집니다.

구현하면,

In [9]:
isSafeRow' tryPos posQueens = snd tryPos `notElem` map snd posQueens
isSafeCol' tryPos posQueens = fst tryPos `notElem` map fst posQueens
isSafeIncCross' tryPos posQueens = uncurry (+) tryPos `notElem` map (uncurry (+)) posQueens        
isSafeDecCross' tryPos posQueens = decStartPos tryPos `notElem` map decStartPos posQueens
    where
        decStartPos posQueens
            | uncurry (<=) posQueens = (0, snd posQueens - fst posQueens)
            | otherwise = (uncurry (-) posQueens, 0)

테스트를 위해 퀸을 둘 수 없는 자리만 뽑아서 정확히 나왔는지 확인해보겠습니다. 

In [10]:
flatmap :: (a1 -> [a]) -> [a1] -> [a]
flatmap f s = foldl1 (++) (map f s)

* 위의 함수 `flatmap`은 nested map을 사용할 때 2중으로 괄호가 씌어지기 때문에( 예를 들면, `[[], []]` 이렇게 ) 이를 다시 평평하게 해주는 기능을 합니다.

In [11]:
testPos = (2,2)
n = 8
allPos = flatmap (\col -> (map (\row -> (row,col)) [0..(n-1)])) [0..(n-1)]

In [12]:
extDangerRowPos n testPos = filter (\x -> not (x `isSafeRow'` [testPos])) allPos
extDangerColPos n testPos = filter (\x -> not (x `isSafeCol'` [testPos])) allPos
extDangerIncCrossPos n testPos = filter (\x -> not (x `isSafeIncCross'` [testPos])) allPos
extDangerDecCrossPos n testPos = filter (\x -> not (x `isSafeDecCross'` [testPos])) allPos

In [13]:
extDangerRowPos n testPos
extDangerColPos n testPos
extDangerIncCrossPos n testPos
extDangerDecCrossPos n testPos

[(0,2),(1,2),(2,2),(3,2),(4,2),(5,2),(6,2),(7,2)]

[(2,0),(2,1),(2,2),(2,3),(2,4),(2,5),(2,6),(2,7)]

[(4,0),(3,1),(2,2),(1,3),(0,4)]

[(0,0),(1,1),(2,2),(3,3),(4,4),(5,5),(6,6),(7,7)]

`(2,2)`를 기준으로 수직, 수평, 대각선 방향으로 모든 좌표가 포함되었음을 알 수 있습니다. (8x8 보드 기준)

방금 만든 기능은 모두 퀸을 둘 수 있는 위치를 판단하는 공통점이 있으므로 `isSafe`라는 함수에 묶어서 정의하겠습니다.

In [14]:
isSafe :: (Num a, Ord a) => (a, a) -> [(a, a)] -> Bool
isSafe tryPos posQueens = isSafeRow && isSafeCol && isSafeIncCross && isSafeDecCross
    where
        isSafeRow = snd tryPos `notElem` map snd posQueens
        isSafeCol = fst tryPos `notElem` map fst posQueens
        isSafeIncCross = uncurry (+) tryPos `notElem` map (uncurry (+)) posQueens        
        isSafeDecCross = decStartPos tryPos `notElem` map decStartPos posQueens
            where
                decStartPos posQueens
                    | uncurry (<=) posQueens = (0, snd posQueens - fst posQueens)
                    | otherwise = (uncurry (-) posQueens, 0)

In [24]:
extDangerPos n testPos = filter (\x -> not (x `isSafe` [testPos])) allPos
    where allPos = flatmap (\col -> (map (\row -> (row,col)) [0..(n-1)])) [0..(n-1)]

아래에 제시된 집합의 성질을 이용해 퀸을 둘 수 없는 각 성분의(수직,수평,대각선) 부분합과 전체 합이 같음을 확인해보면,

`if A - B = B - A then A = B Where ` $A = a_0 ∪ a_1 ∪ ... ∪ a_n$


In [33]:
import Data.List


unionDangerPos = (
    extDangerRowPos n testPos `union` extDangerColPos n testPos) `union`
    (extDangerIncCrossPos n testPos `union` extDangerDecCrossPos n testPos)
    
unionDangerPos \\ extDangerPos n testPos ==
    extDangerPos n testPos \\ unionDangerPos

True

백트래킹은 메아리와 비슷한 원리

$Depth_{Width}$

1. Depth Search :  $a_0 -> b_0 -> .. -> (n-1)_0$

2. Width Search : $-> (n-1)_1 -> ... -> (n-1)_{n-1} -> (n-2)_{1} -> ... -> (n-2)_{n-1} -> ... -> a_1 -> a_2 -> ... -> a_{n-1}$


깊이를 우선으로 탐색하되, 적절한Safe 위치가 없으면 넓이를 탐색하도록 하겠습니다.


In [34]:
queenPuzzle n = _queenPuzzle [] 0 0
    where
        _queenPuzzle posQueens row col
            | row == n = posQueens
            | isSafePos = _queenPuzzle ((row,col):posQueens) 0 (col+1)
            | otherwise = _queenPuzzle posQueens (row+1) col
                where
                    isSafePos = isSafe (row,col) posQueens

In [37]:
queenPuzzle 5

[(3,4),(1,3),(4,2),(2,1),(0,0)]